**Libraries**


In [1]:
!pip install Biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.6 MB/s eta 0:00:00


In [2]:
import argparse
import pandas as pd
import re
from Bio import Entrez

**This function fetches PubMed articles based on a search query using Entrez, returning a list of articles in XML format.**





In [3]:
def fetch_pubmed_papers(query, max_results=50):
    """Fetches papers from PubMed based on the query."""
    Entrez.email = "bhavyasri@example.com"

    try:
        handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results)
        record = Entrez.read(handle)
        pmid_list = record["IdList"]

        if not pmid_list:
            print("No results found for the given query.")
            return []

        handle = Entrez.efetch(db="pubmed", id=",".join(pmid_list), retmode="xml")
        papers = Entrez.read(handle)
        return papers["PubmedArticle"]
    except Exception as e:
        print(f"Error fetching data: {e}")
        return []

**This function extracts metadata from PubMed articles, filters for non-academic authors affiliated with companies, and captures corresponding author emails.**

In [4]:
def extract_paper_info(papers):
    """Extracts necessary metadata and filters for non-academic authors."""
    results = []

    for paper in papers:
        try:
            PubmedID= paper["MedlineCitation"]["PMID"]
            Title= paper["MedlineCitation"]["Article"]["ArticleTitle"]
            Publication_Date= paper["MedlineCitation"]["Article"]["Journal"]["JournalIssue"]["PubDate"]
            Non_academic_Authors= paper["MedlineCitation"]["Article"].get("AuthorList", [])

            Publication_Date= " ".join([str(Publication_Date.get(k, "")) for k in ["Year", "Month", "Day"] if Publication_Date.get(k)])

            Non_academic_authors = []
            Company_affiliations = []
            Corresponding_Author_email = None

            for author in Non_academic_Authors:
                if "AffiliationInfo" in author:
                    for aff in author["AffiliationInfo"]:
                        affiliation = aff["Affiliation"]

                        if re.search(r"\b(pharma|biotech|laboratories|inc|corp|gmbh|ltd|llc|co.)\b", affiliation, re.I):
                            Non_academic_authors.append(f"{author['ForeName']} {author['LastName']}")
                            Company_affiliations.append(affiliation)

                        email_match = re.search(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", affiliation)
                        if email_match and not Corresponding_Author_email:
                            Corresponding_Author_email = email_match.group()

            if Company_affiliations:
                results.append([PubmedID, Title, Publication_Date, ", ".join(Non_academic_authors), ", ".join(Company_affiliations), Corresponding_Author_email])

        except Exception as e:
            print(f"Error processing a paper: {e}")

    return results

**This function saves the extracted PubMed article data to a CSV file.**


In [5]:
def save_to_csv(data, filename):
    """Saves extracted data to a CSV file."""
    df = pd.DataFrame(data, columns=["PubmedID", "Title", "Publication Date", "Non-academic Authors", "Company Affiliations", "Corresponding Author Email"])
    df.to_csv(filename, index=False)
    print(f"Results saved as {filename}")

**This function processes command-line arguments, fetches PubMed papers, extracts relevant data, and either saves it to a CSV file or prints it to the console.**


In [6]:
import argparse

def main():
    parser = argparse.ArgumentParser(description="Fetch PubMed papers with non-academic authors.")

    # Accepts the PubMed query string
    parser.add_argument("--query", "-q", required=True, help="PubMed query string")

    # Corrected file output option
    parser.add_argument("--file", "-f", help="Output CSV file name (if not provided, results are printed to console)")

    # Debug mode option
    parser.add_argument("--debug", "-d", action="store_true", help="Enable debug mode")

    args = parser.parse_args()

    if args.debug:
        print("[DEBUG] Debug mode enabled.")

    papers = fetch_pubmed_papers(args.query, max_results=50)
    filtered_data = extract_paper_info(papers)

    if len(filtered_data) < 10:
        print(f"Warning: Only {len(filtered_data)} results found. Try adjusting the query.")
    else:
        print(f"Successfully retrieved {len(filtered_data)} papers.")

    # Save to CSV or print results
    if args.file:
        save_to_csv(filtered_data, args.file)
    else:
        print("\n--- Results ---\n")
        for row in filtered_data:
            print(row)



**This script detects if it's running in Google Colab, runs a predefined PubMed query, saves results to a CSV, or executes the CLI-based `main()` function otherwise.**


In [7]:
if __name__ == "__main__":
    import sys

    # Detect if running inside Google Colab
    if "google.colab" in sys.modules:
        query = "biotech OR pharma"  # Modify this query as needed
        output_file = "pubmed_results.csv"

        print(f"Running in Google Colab with query: {query}")
        papers = fetch_pubmed_papers(query, max_results=50)
        filtered_data = extract_paper_info(papers)
        save_to_csv(filtered_data, output_file)
    else:
        main()

Running in Google Colab with query: biotech OR pharma
Results saved as pubmed_results.csv


**This code loads the `pubmed_results.csv` file into a pandas DataFrame and displays the first five rows.**


In [9]:
import pandas as pd

df = pd.read_csv("pubmed_results.csv")
df.head()  # Display first 5 rows

,PubmedID,Title,Publication Date,Non-academic Authors,Company Affiliations,Corresponding Author Email
0,40066684,Mechanistic Investigation into the Phase Separ...,2025 Mar 11,"Malte Bøgh Senniksen, Nicole Wyttenbach, Susan...","Pharmaceutical R&D, F. Hoffmann-La Roche Ltd.,...",NaN
1,40066659,Inhibition of HDAC4 in granulosa cells improve...,2025 Mar 11,"Zhenzi Zuo, Yibing Bao, Lin Lin, Hengxing Li, ...",State Key Laboratory of Animal Biotech Breedin...,NaN
2,40066624,Enzyme cycling method using hypoxanthine-guani...,2025 Mar 11,"Tatsuya Hirata, Shin-Ichi Sakasegawa","R&D Group, Diagnostics Dept., Asahi Kasei Phar...",shigeru-ueda@jccls.org
3,40066621,Lyophilized monkeypox mRNA lipid nanoparticle ...,2025 Dec,"Bin Wang, Quanyi Yin, Li Yi, Caixia Su, Yang W...","Department of Research & Development, Yither B...",NaN
4,40066446,Exercise boost after surgery improves survival...,2025,"Rikke Stagaard, Adina Jensen, Jan Erik Strøbec...","Biotech Research and Innovation Center (BRIC),...",NaN
